This notebook was automatically generated from your MAST-ML run so you can recreate the
plots. Some things are a bit different from the usual way of creating plots - we are
using the [object oriented
interface](https://matplotlib.org/tutorials/introductory/lifecycle.html) instead of
pyplot to create the `fig` and `ax` instances.


In [ ]:
"""
This module contains a collection of functions which make plots (saved as png files) using matplotlib, generated from
some model fits and cross-validation evaluation within a MAST-ML run.

This module also contains a method to create python notebooks containing plotted data and the relevant source code from
this module, to enable the user to make their own modifications to the created plots in a straightforward way (useful for
tweaking plots for a presentation or publication).
"""

import math
import os
import pandas as pd
import itertools
import warnings
import logging
from collections import Iterable
from os.path import join
from collections import OrderedDict

# Ignore the harmless warning about the gelsd driver on mac.
warnings.filterwarnings(action="ignore", module="scipy",
                        message="^internal gelsd")
# Ignore matplotlib deprecation warning (set as all warnings for now)
warnings.filterwarnings(action="ignore")

import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure, figaspect
from matplotlib.animation import FuncAnimation
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

# Needed imports for ipynb_maker
from mastml.utils import nice_range
from mastml.metrics import nice_names

import inspect
import textwrap
from pandas import DataFrame, Series

import nbformat

from functools import wraps

matplotlib.rc('font', size=18, family='sans-serif') # set all font to bigger
matplotlib.rc('figure', autolayout=True) # turn on autolayout

# adding dpi as a constant global so it can be changed later
DPI = 250

log = logging.getLogger() # only used inside ipynb_maker I guess



In [ ]:
def stat_to_string(name, value):
    """
    Method that converts a metric object into a string for displaying on a plot

    Args:

        name: (str), long name of a stat metric or quantity

        value: (float), value of the metric or quantity

    Return:

        (str), a string of the metric name, adjusted to look nicer for inclusion on a plot

    """

    " Stringifies the name value pair for display within a plot "
    if name in nice_names:
        name = nice_names[name]
    else:
        name = name.replace('_', ' ')

    # has a name only
    if not value:
        return name
    # has a mean and std
    if isinstance(value, tuple):
        mean, std = value
        return f'{name}:' + '\n\t' + f'{mean:.3f}' + r'$\pm$' + f'{std:.3f}'
    # has a name and value only
    if isinstance(value, int) or (isinstance(value, float) and value%1 == 0):
        return f'{name}: {int(value)}'
    if isinstance(value, float):
        return f'{name}: {value:.3f}'
    return f'{name}: {value}' # probably a string


def plot_stats(fig, stats, x_align=0.65, y_align=0.90, font_dict=dict(), fontsize=14):
    """
    Method that prints stats onto the plot. Goes off screen if they are too long or too many in number.

    Args:

        fig: (matplotlib figure object), a matplotlib figure object

        stats: (dict), dict of statistics to be included with a plot

        x_align: (float), float denoting x position of where to align display of stats on a plot

        y_align: (float), float denoting y position of where to align display of stats on a plot

        font_dict: (dict), dict of matplotlib font options to alter display of stats on plot

        fontsize: (int), the fontsize of stats to display on plot

    Returns:

        None

    """

    stat_str = '\n'.join(stat_to_string(name, value)
                           for name,value in stats.items())

    fig.text(x_align, y_align, stat_str,
             verticalalignment='top', wrap=True, fontdict=font_dict, fontproperties=FontProperties(size=fontsize))


def make_fig_ax(aspect_ratio=0.5, x_align=0.65, left=0.10):
    """
    Method to make matplotlib figure and axes objects. Using Object Oriented interface from https://matplotlib.org/gallery/api/agg_oo_sgskip.html

    Args:

        aspect_ratio: (float), aspect ratio for figure and axes creation

        x_align: (float), x position to draw edge of figure. Needed so can display stats alongside plot

        left: (float), the leftmost position to draw edge of figure

    Returns:

        fig: (matplotlib fig object), a matplotlib figure object with the specified aspect ratio

        ax: (matplotlib ax object), a matplotlib axes object with the specified aspect ratio

    """
    # Set image aspect ratio:
    w, h = figaspect(aspect_ratio)
    fig = Figure(figsize=(w,h))
    FigureCanvas(fig)

    # Set custom positioning, see this guide for more details:
    # https://python4astronomers.github.io/plotting/advanced.html
    #left   = 0.10
    bottom = 0.15
    right  = 0.01
    top    = 0.05
    width = x_align - left - right
    height = 1 - bottom - top
    ax = fig.add_axes((left, bottom, width, height), frameon=True)
    fig.set_tight_layout(False)
    
    return fig, ax


In [ ]:
def plot_predicted_vs_true(train_quad, test_quad, outdir, label):
    """
    Method to create a parity plot (predicted vs. true values)

    Args:

        train_quad: (tuple), tuple containing 4 numpy arrays: true training y data, predicted training y data,

        training metric data, and groups used in training

        test_quad: (tuple), tuple containing 4 numpy arrays: true test y data, predicted test y data,

        testing metric data, and groups used in testing

        outdir: (str), path to save plots to

        label: (str), label used for axis labeling

    Returns:

        None

    """
    filenames = list()
    y_train_true, y_train_pred, train_metrics, train_groups = train_quad
    y_test_true, y_test_pred, test_metrics, test_groups = test_quad

    # make diagonal line from absolute min to absolute max of any data point
    # using round because Ryan did - but won't that ruin small numbers??? TODO this
    max1 = max(y_train_true.max(), y_train_pred.max(),
               y_test_true.max(), y_test_pred.max())
    min1 = min(y_train_true.min(), y_train_pred.min(),
               y_test_true.min(), y_test_pred.min())
    max1 = round(float(max1), rounder(max1-min1))
    min1 = round(float(min1), rounder(max1-min1))
    for y_true, y_pred, stats, groups, title_addon in \
            (train_quad+('train',), test_quad+('test',)):

        # make fig and ax, use x_align when placing text so things don't overlap
        x_align=0.64
        fig, ax = make_fig_ax(x_align=x_align)

        # set tick labels
        # notice that we use the same max and min for all three. Don't
        # calculate those inside the loop, because all the should be on the same scale and axis
        _set_tick_labels(ax, max1, min1)

        # plot diagonal line
        ax.plot([min1, max1], [min1, max1], 'k--', lw=2, zorder=1)

        # do the actual plotting
        if groups is None:
            ax.scatter(y_true, y_pred, color='blue', edgecolors='black', s=100, zorder=2, alpha=0.7)
        else:
            handles = dict()
            unique_groups = np.unique(np.concatenate((train_groups, test_groups), axis=0))
            log.debug(' '*12 + 'unique groups: ' +str(list(unique_groups)))
            colors = ['blue', 'red', 'green', 'purple', 'orange', 'black']
            markers = ['o', 'v', '^', 's', 'p', 'h', 'D', '*', 'X', '<', '>', 'P']
            colorcount = markercount = 0
            for groupcount, group in enumerate(unique_groups):
                mask = groups == group
                log.debug(' '*12 + f'{group} group_percent = {np.count_nonzero(mask) / len(groups)}')
                handles[group] = ax.scatter(y_true[mask], y_pred[mask], label=group, color=colors[colorcount],
                                            marker=markers[markercount], s=100, alpha=0.7)
                colorcount += 1
                if colorcount % len(colors) == 0:
                    markercount += 1
                    colorcount = 0
            ax.legend(handles.values(), handles.keys(), loc='lower right', fontsize=12)

        # set axis labels
        ax.set_xlabel('True '+label, fontsize=16)
        ax.set_ylabel('Predicted '+label, fontsize=16)

        plot_stats(fig, stats, x_align=x_align, y_align=0.90)

        filename = 'predicted_vs_true_'+ title_addon + '.png'
        filenames.append(filename)
        fig.savefig(join(outdir, filename), dpi=DPI, bbox_inches='tight')

    return filenames


In [ ]:
from numpy import array
from collections import OrderedDict
from io import StringIO
train_quad = (array([150.37102595, 202.0924573 , 325.49788774, ...,  19.41019935,
         0.        ,  47.20790294]), array([7.51855130e+01, 1.01046229e+02, 1.62748964e+02, ...,
       9.70509967e+00, 1.25861019e-06, 2.36039515e+01]), OrderedDict([('R2', 0.9988855555310615), ('root_mean_squared_error', 71.24016596101944), ('mean_absolute_error', 52.94675284772316), ('rmse_over_stdev', 0.7443155035785749)]), None)
test_quad = (array([177.28302038, 249.54944341, 210.37520634, 200.26888144,
       239.2147742 , 296.50121028, 276.21736875, 193.72169364,
       193.03789201, 396.88333541,   0.        ,  64.76678569,
       146.59493555, 143.14427   , 129.67655739, 185.18233461,
       176.19335258,  67.39021938, 113.27094167, 149.58971125,
       117.55701155,  85.32116056,  97.75371527,  88.06968527,
        89.11498671, 233.37177602, 181.75592179, 171.80996061,
       181.77618797, 172.42020306, 178.66988561, 125.7729607 ,
       369.78013418, 306.66689715, 335.55187932, 124.02294655,
       119.65758775, 116.8908945 , 127.740217  ,  80.94033473,
       187.42767666, 158.36740952, 291.95555798,  45.7531836 ,
        24.89260867,  87.31329665, 120.21481561, 270.00791293,
       106.61645203, 156.22972982,  71.90056154,  86.43036922,
        84.20250562, 113.13530223, 201.81756512, 198.06185912,
        73.49782973, 163.29786494,  69.43614177,  92.45610791,
        34.7342257 ,  29.45910878, 158.21863233,  30.40327127,
       114.52532705,  36.52821981,  50.0993    ,   0.        ,
         0.        ,   0.        ,   0.        , 307.83053925,
       140.13445715,  29.91808225,  51.86444618, 112.0035505 ,
       218.9463975 , 145.19783167, 202.6896798 , 219.40213351,
       117.65052986,   0.        ,  34.47065573,  60.12311255,
         0.        ,  29.94908075,  55.71532531,  25.83894891,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   1.92094295,   0.        ,
         0.        ,  27.03043917,  12.63281495,  11.84797082,
        16.57505557,   1.747132  ,  56.75173357,  44.85051669,
        48.99432914,  43.99569352,  39.22644569, 162.30858215,
       133.55965834, 140.19831868, 150.50818565,  86.40676351,
       115.20029363,   0.        ,  30.46897905,   0.        ,
        29.653255  ,   0.        ,   9.52352009,   0.        ,
         0.        ,  13.80460838,  52.62963569,  34.51982144,
        62.11081062, 118.54297288,  28.81237225,  24.13350925,
        95.52052281,   0.        ,  47.61927987,  35.46209575,
        26.16951837,  48.57118688,  73.3952221 , 117.71448387,
        36.257048  , 157.59559963, 100.85978128,  85.19420825,
        75.88549386, 187.39104625,   0.        , 137.12605894,
       170.3886173 ,   0.        ,   0.        ,  69.47351553,
       284.89818957,  44.54144834,  99.7625282 , 185.52454231,
        52.02867683, 202.66455344, 140.53855037, 134.5064424 ,
       139.34508133,  49.69176246,  73.13416251,  57.38221378,
        47.801376  ,  38.14651548,  36.16956355,   0.        ,
        26.23033875,  32.97595651,  22.41961241,   0.        ,
         0.        ,  55.47285962,   0.        ,   0.        ,
        44.84888737,  48.37559255,  58.54651811,  17.0719435 ,
         0.        ,  47.54553553,  91.38113959,  74.56373534,
       135.99188516,  56.54071259,  58.73687405,  58.3323071 ,
        76.83212346,  50.298441  ,  37.03306019,  42.81588044,
        34.01111394,  31.35144814,   0.        ,   0.        ,
        65.03154095,  18.9559087 , 134.6716424 ,  91.34499746,
        60.72512971,   0.        ,  65.98496318,  94.30190586,
        36.18162447,  26.50926775,  26.74964638,  42.90406645,
        10.35277113,  28.72421875,  33.67375834,  21.819592  ,
        66.76133312,  95.22684338,  42.34404359,   0.        ,
        14.00114963,   0.        ,   5.43550612,  35.72806622,
       178.13772044,  23.22818913,   7.05592531,   9.36490231,
       103.74396875, 133.03032487, 142.33395365, 157.24298149,
       245.69243888, 250.25203832, 260.12973722, 207.90480672,
       234.66065082, 275.65617815,  99.68134329, 187.87580019,
       252.76716229, 243.428371  , 224.62287057, 153.2465035 ,
       186.02653043, 114.50715188, 130.34718776, 114.4981617 ,
       122.77257944, 130.39035205, 189.88100424, 170.62748097,
       292.39066312, 100.86488318, 123.24662923, 122.43167022,
       205.09292614, 191.51236301, 227.87867171, 201.32142318,
       111.02632779, 129.31911366,  76.97840947, 197.12412955,
       195.67275424,  91.28475717,  88.62675986, 161.29335072,
       247.00509507,  96.92679621, 192.08874679, 198.5926738 ,
       141.92110132,  94.43350892, 234.968862  , 216.45461264,
       313.72288064, 160.23398776, 259.08638129, 354.73775188,
       353.34957447, 179.99933217, 317.205543  , 328.54172252,
       346.90203425, 164.00223776, 120.40352958, 141.45345548,
        68.40090273,  55.94984366,  89.54726266, 244.80034507,
       151.8983111 , 165.1097297 ,  64.5557199 , 158.20291287,
        77.46656255, 133.88336745, 132.39243296,  53.12824467,
       158.4134052 , 238.53505817, 324.94508429, 148.34054822,
       313.927543  , 228.16969945, 347.29590062, 857.23166531,
       142.41612109, 105.93237224,  95.77864058,  78.21525252,
        51.98836201,  77.08989732, 190.04640497,  94.39134167,
        96.47357254,  39.04655223,  17.43031391,  35.10096061,
        60.78290984, 113.32195047,  49.6708469 , 112.90930286,
       127.66464503,  68.87735374,  30.53286251, 133.6354298 ,
        58.54469733,  27.21757246, 187.84859075, 135.8400834 ,
       121.55357738, 211.33526584, 184.77929222, 163.40928559,
       115.40786595, 179.71035609, 137.40668645,  33.28049783,
        77.38602769,  83.56862417,  90.60897065, 174.06027747,
       113.09210788, 183.15120004, 397.23996543, 237.58337883,
       117.14051442, 138.58290334, 140.36478149, 144.11116592,
       244.71754232,  88.68589559, 263.54337143,  44.18158824,
        28.61719851,  46.91723712,   4.45796202,  38.63286983,
       165.73307859,  72.67526425,  61.69987974, 341.670362  ,
         0.        ,   0.        , 109.97295248, 161.96626836,
       179.36951362, 182.44993004, 110.11022338,  84.58123286,
         0.        ,  66.26244234,  82.91347525,   6.92239663,
         0.        ,  21.50586649,   4.75557533,  28.36967345,
         0.        ,  61.55780711,  28.48129225,   0.        ,
         0.        ,  84.72896518,  45.6348093 ,  27.12929863,
        14.17874738,  26.70381011,  33.75589221,  37.46802334,
        81.726855  ,  68.97106961, 166.79781986,  52.14150703,
         0.        ,  55.08861603,  45.03777853,  44.40936128,
        48.24004019,  38.17424353,  33.9806022 ,  43.94549002,
         0.        , 190.87087845,  82.62024503,  82.58263368,
        75.67939565, 157.61174657,  70.94302761,  36.22347961,
         0.        ,  90.16563994, 126.61616369,  28.07241153,
        30.74580969, 120.86584011,  29.23073416,  36.61104469,
        42.696629  ,  70.04323975,  17.00030381,  52.42546125,
        46.69821725,  33.46241925,  27.90730325, 121.7771504 ,
        38.754046  ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,  33.52619036,  23.809066  ,
         0.        ]), array([ 0.00000000e+000,  0.00000000e+000,  8.65474225e-236,
        3.17688354e-143,  1.48720127e-005,  4.92516655e-073,
        1.99520369e-005,  0.00000000e+000,  0.00000000e+000,
        9.12890538e-123,  2.09731794e-304,  3.56258670e-028,
        8.78222560e-135,  1.61365538e-274,  0.00000000e+000,
        6.90190667e-192,  4.54035583e-028,  1.68120307e-027,
        7.13326363e-135,  2.70639713e-238,  1.31304824e-027,
        2.40698312e-231,  0.00000000e+000,  4.35093454e-192,
        2.33279857e-231,  3.02089960e-027,  7.31395946e-133,
        1.44584136e-132,  4.78902131e-231,  0.00000000e+000,
        1.54660104e-122,  0.00000000e+000,  3.24338957e-010,
        4.18863771e-236,  4.88527188e-236,  8.42027393e-148,
        2.71521995e-238,  1.90685154e-252,  1.59735846e-132,
        0.00000000e+000,  7.91999927e-133,  0.00000000e+000,
        6.01025561e-306,  1.61741939e-008,  1.18341055e+002,
        2.04415913e-023,  1.84924002e-023,  5.98787334e-111,
        3.98108994e-062,  1.08192522e-003,  3.66921215e-022,
        0.00000000e+000,  2.53570184e+000,  1.84375346e-010,
        4.69569710e-045,  7.74563193e-002,  1.27840792e-013,
        0.00000000e+000,  1.40573846e-038,  0.00000000e+000,
        0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
        2.63162579e-076,  0.00000000e+000,  4.58311387e-026,
        0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
        0.00000000e+000, -2.49573433e-095,  1.31274761e-036,
        6.94277492e-024,  0.00000000e+000,  3.50877210e-011,
        5.58052722e-032,  8.43881017e-002,  1.12005961e-005,
        1.61859187e-025,  4.42833004e-009,  2.46017823e-084,
        0.00000000e+000,  6.71252571e-237,  2.13409326e-238,
        0.00000000e+000,  5.38578288e-002,  7.62153991e-009,
        0.00000000e+000,  0.00000000e+000,  2.20151291e-240,
        4.25818944e-013,  1.15051057e-022,  0.00000000e+000,
        0.00000000e+000,  1.06344793e-004,  3.55533284e-003,
        0.00000000e+000, -9.20190059e-009,  2.65089394e-001,
        6.17876902e-066,  4.36505558e-029,  1.20974745e-243,
        4.50587078e-012,  2.48428775e-006,  7.01979259e-002,
        4.06135176e+000,  1.23553664e-253,  8.71218855e-084,
        2.25642071e+000,  7.98687675e-236,  5.28439161e-234,
        2.31329464e-001,  2.95460155e+000,  0.00000000e+000,
        6.66263058e-095,  5.44040604e-135,  6.98041852e-005,
        5.58130611e-240,  1.53700735e-277,  9.68352000e-245,
        6.46174681e-026,  6.11455022e-085,  2.29263913e-010,
        7.88607332e-005,  3.83666377e-012,  1.61544247e-004,
        1.12890500e-047,  8.19500489e-013,  1.36657786e-003,
        0.00000000e+000,  3.27548944e-012, -1.29689287e-201,
        7.65276087e-007,  2.42839090e+000,  4.66533338e-015,
        3.89569346e-006,  0.00000000e+000,  0.00000000e+000,
        0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
        4.20930396e-094,  1.58752767e-025,  3.05833775e-025,
        0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
        0.00000000e+000,  4.85646573e-011,  0.00000000e+000,
        1.08564608e-036,  9.25169804e-009,  1.54884335e-113,
        5.18268039e-038,  6.96367387e-011,  1.33600633e-183,
        7.01698071e-099,  1.34262613e-236,  3.71371796e-234,
        6.13489446e-008,  5.87340747e-004,  6.64864309e-077,
        1.02480376e+000,  0.00000000e+000,  5.63770956e-002,
        5.94226859e-002,  0.00000000e+000,  3.12376531e-007,
        1.35876245e-008,  1.23272672e-101,  0.00000000e+000,
        0.00000000e+000,  1.13127508e+001,  0.00000000e+000,
        0.00000000e+000,  5.50558486e-003,  0.00000000e+000,
        1.55955593e+000,  9.92080963e-008,  1.62467033e+001,
        2.07090922e-076,  1.29362111e-006,  4.24615190e-007,
        1.44113155e-069,  4.71262747e-012,  0.00000000e+000,
        5.69754909e-102,  6.99364041e-004,  2.87769506e-077,
        6.19451992e-003,  0.00000000e+000,  0.00000000e+000,
        9.85182618e-238,  1.36258568e-239,  2.15132452e-003,
        3.02128040e-239,  5.24409147e-237,  1.20461129e-237,
        0.00000000e+000,  0.00000000e+000,  1.47945386e-006,
        1.97722279e-002,  4.79804004e-004,  6.97671932e-071,
        2.90148412e-009,  1.43538798e-069,  5.94390446e-017,
        3.30485363e-102,  9.66402810e-133,  5.61570617e-133,
        4.10594534e-002,  0.00000000e+000,  5.76328059e-039,
        5.63306841e-013,  1.19304320e-118,  1.14807147e-016,
        7.06669786e-007,  1.54574877e-070,  3.74676751e-134,
        8.36442046e-199,  4.72606739e-027,  2.92146476e+000,
        2.49358158e-001,  2.78176722e-005,  1.17964580e-064,
        2.63263969e-043,  7.67485974e-015,  6.39647127e-020,
        4.63950126e-008,  0.00000000e+000,  2.23590794e-008,
        2.73759816e-004,  5.12990949e-001,  6.45845271e-010,
        0.00000000e+000,  3.30671408e+000,  4.75718022e+000,
        2.77563440e+000,  3.19311664e+000,  2.15298767e-001,
        2.39494413e-001,  2.43789594e-001,  4.17575232e-008,
        3.30499873e-001,  7.32359585e-002,  2.35985226e-001,
        1.88713422e-001,  1.30444694e+001,  2.14114720e+001,
        4.21432651e-002,  5.17668837e+000,  4.75000941e+000,
        2.47148335e+000,  1.45011146e+000,  1.85576031e+000,
        3.47558127e-001,  3.58344729e-020,  2.30289164e-008,
        5.30661876e-001,  2.86524080e-001,  6.24576203e-002,
        1.92623836e-001,  3.73189617e-001,  3.97496259e-001,
        2.97544815e-001,  9.56499206e+000,  4.74543582e+000,
        4.38684218e+000,  6.61928239e-004,  6.58324834e-004,
        6.23013437e-004,  7.74184245e-001,  7.19513271e-001,
        4.41224446e-002,  8.60741189e+000,  1.00964981e+001,
        7.08435028e+000,  6.58415769e-004,  3.00230726e+000,
        3.46535214e+000,  5.05274135e-021,  5.99010371e-009,
        2.51113595e-001,  6.16112413e-002,  2.67890326e-001,
        3.13215230e-001,  1.23145525e-001,  1.06695373e-001,
        1.85678415e-008,  1.27806067e+001,  1.30272279e+001,
        4.56788441e+000,  4.09992564e+000,  3.60819896e-004,
        6.62534298e-001,  3.67086097e-002,  8.52551213e+000,
        7.00720751e+000,  1.48762761e-032,  3.45657176e-010,
        2.74472045e-006,  5.89323977e-017,  2.92779222e-025,
        2.31357347e-001,  2.01462217e-017,  2.43255016e-025,
        1.38501877e-003,  7.94668728e-004,  7.42015568e-004,
        9.32775615e-011,  3.78803850e-027,  6.66456239e-110,
        3.50897491e-006,  5.79759891e-060,  9.55318465e-027,
        1.68088907e-109,  1.02373369e-003,  4.44069625e-001,
        4.42109549e-001,  9.73240481e-004,  4.30208721e-001,
        1.60822042e-004,  3.66353362e-003,  6.70679218e-234,
        2.07232087e-103,  1.18534786e-063,  1.04203889e-038,
        4.01021233e-003,  1.05115172e-114,  4.68837805e+000,
        1.30607836e-158,  5.31517796e-028,  0.00000000e+000,
        2.73880379e+001,  1.35779179e+002,  1.82865959e-003,
        4.74845628e+000,  1.02594550e-003,  0.00000000e+000,
        6.00189415e+000,  3.89518822e+000,  9.30862350e-005,
        3.44457242e+000,  1.93981289e+001,  2.86591203e-003,
        2.12592735e+000,  0.00000000e+000,  7.45313741e-026,
        1.12454428e-023,  3.35609950e-027,  2.85931610e-019,
        1.87312748e-008,  3.00879212e-005,  8.93446239e-013,
        9.47225088e-003,  1.67808478e-037,  4.13238490e-026,
        6.44869797e-235,  7.66657145e-006,  9.22221807e-027,
        2.60273026e-090,  0.00000000e+000,  9.94389891e-099,
        7.56658961e-024,  0.00000000e+000,  7.38237276e-003,
        4.58810280e-031,  0.00000000e+000,  0.00000000e+000,
        2.10866561e+000,  1.82245858e-006,  4.08312119e-004,
        0.00000000e+000,  1.10190048e-001,  2.82124452e-028,
        0.00000000e+000,  0.00000000e+000,  3.06115801e-080,
        6.20529438e-078,  4.50329632e-028,  3.07585248e-003,
        1.11220917e-005,  3.32925773e-002,  1.67364651e+000,
        9.99141628e-028,  1.48928462e-006,  3.33584392e-033,
        6.00981790e-028,  0.00000000e+000,  6.37423361e-031,
        6.26002322e-011,  1.11722543e-017,  3.26007360e-063,
       -3.99281035e-031, -4.74560682e-019,  7.38263966e-003,
        0.00000000e+000,  3.89135007e-080,  1.04542607e-010,
        0.00000000e+000,  0.00000000e+000,  7.66173998e-063,
        1.13130783e-001,  8.89038159e-004,  0.00000000e+000,
        7.01143891e-001,  2.35277813e-080,  2.59652151e-019,
        4.50057844e-011,  4.98733288e-063,  4.82107672e-009,
        2.32430964e-157,  1.39789763e-069,  1.69081890e-080,
        0.00000000e+000,  7.28800816e-031,  6.43694779e-011,
        1.42016630e-063,  6.36433324e-013,  9.78739117e-007,
        0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
       -1.44023007e-127,  0.00000000e+000,  5.75215479e-003,
        2.31638325e-003,  3.01387771e-027,  0.00000000e+000]), OrderedDict([('R2', 0.00014118866704371236), ('root_mean_squared_error', 138.88380882420793), ('mean_absolute_error', 102.50104921458386), ('rmse_over_stdev', 1.4510546221981537)]), None)
outdir = '/home/nerve/Desktop/skunkworks/lane_schultz/hyper_parameters/test/DoNothing/DoNothing/KernelRidge/RepeatedKFold/split_2'
label = 'Energy above convex hull (meV/atom)'

In [ ]:
import pandas as pd
from IPython.display import Image, display

plot_paths = plot_predicted_vs_true(train_triple, test_triple, outdir)
for plot_path in plot_paths:
    display(Image(filename=plot_path))
